In [1]:
pip install binance pandas


Note: you may need to restart the kernel to use updated packages.


In [24]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import os
import time

# Initialize the Binance client
api_key = 'dd1a2703da01c24ccdbe10b2cbc9600a6218c41434d4453916766e1e817aabfe'
api_secret = '2acb5c37db3c5646fdadb366cacb5d9a11ffe05d7b4d01dfb1b3047f2a2ea09a'
client = Client(api_key, api_secret)
client.API_URL = 'https://testnet.binance.vision/api'  # Use testnet for sandbox mode


In [25]:
# Path to the CSV file
csv_file_path = 'ohlc_1m_data_last_day.csv'


In [26]:
def get_last_timestamp(csv_file_path):
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path, parse_dates=['timestamp'])
        if not df.empty:
            return int(df['timestamp'].max().timestamp() * 1000)
    return int(datetime.strptime("2024-07-04 00:00:00", "%Y-%m-%d %H:%M:%S").timestamp() * 1000)


In [27]:
def fetch_ohlcv_data(symbol, interval, start_str):
    klines = client.get_historical_klines(symbol, interval, start_str)
    return klines


In [28]:
def ohlcv_to_dataframe(data):
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    return df

In [29]:
import pandas as pd
def update_csv():
    last_timestamp_ms = get_last_timestamp(csv_file_path)
    last_timestamp_str = datetime.utcfromtimestamp(last_timestamp_ms / 1000).strftime('%d %b %Y %H:%M:%S')

    symbol = 'BTCUSDT'
    interval = Client.KLINE_INTERVAL_1MINUTE

    # Fetch new data
    new_data = fetch_ohlcv_data(symbol, interval, last_timestamp_str)
    new_df = ohlcv_to_dataframe(new_data)

    # Load existing CSV data
    if os.path.exists(csv_file_path):
        existing_df = pd.read_csv(csv_file_path, parse_dates=['timestamp'])
    else:
        existing_df = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Append new data to the existing DataFrame
    updated_df = pd.concat([existing_df, new_df]).drop_duplicates(subset='timestamp').sort_values(by='timestamp')

    # Save the updated DataFrame to CSV
    updated_df.to_csv(csv_file_path, index=False)
    
    print(f"Data has been updated and saved to {csv_file_path}")


In [30]:
# Custom loop to periodically update CSV
while True:
    update_csv()
    time.sleep(60)  # Run every 1 minute


Data has been updated and saved to ohlc_1m_data_last_day.csv
Data has been updated and saved to ohlc_1m_data_last_day.csv
